# ETL 설계 3편: Silver 레이어 - Bronze를 분석 가능한 상태로 올린다

Bronze에 쌓아둔 원본 데이터를 정제하고 표준화한다. dbt로 변환 로직을 SQL 모델로 정의한다.

📖 [블로그 원문](https://biz-agentic-ai.github.io/guides/etl-design/003-silver-layer/)

## 환경 세팅

In [ ]:
# 도구 설치 (protobuf 경고는 무시해도 됩니다 - Colab의 tensorflow와 버전 충돌이지만 실습에 영향 없음)
!pip install -q duckdb dbt-core dbt-duckdb 2>&1 | grep -v "protobuf"

## Bronze 레이어 준비

1편에서 만든 Bronze 데이터를 다시 적재한다.

In [ ]:
import duckdb

conn = duckdb.connect('warehouse.duckdb')

conn.execute("""
CREATE SCHEMA IF NOT EXISTS bronze;

CREATE OR REPLACE TABLE bronze.orders AS
SELECT *, current_timestamp AS _loaded_at
FROM read_csv_auto('https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_orders.csv');

CREATE OR REPLACE TABLE bronze.customers AS
SELECT *, current_timestamp AS _loaded_at
FROM read_csv_auto('https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_customers.csv');

CREATE OR REPLACE TABLE bronze.payments AS
SELECT *, current_timestamp AS _loaded_at
FROM read_csv_auto('https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_payments.csv');
""")

print("Bronze 적재 완료")
conn.close()

## Bronze 데이터의 문제 확인

Bronze 데이터를 그대로 분석에 쓸 수 없는 이유를 직접 확인한다.

In [ ]:
conn = duckdb.connect('warehouse.duckdb')

# order_date의 타입 확인 — VARCHAR로 들어와 있다
conn.execute("""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'bronze' AND table_name = 'orders'
""").fetchdf()

In [ ]:
# payments의 amount — 센트 단위 정수
conn.execute("SELECT * FROM bronze.payments LIMIT 5").fetchdf()

In [ ]:
conn.close()

## dbt 프로젝트 생성

dbt 프로젝트 디렉토리와 설정 파일을 만든다.

In [ ]:
import os

os.makedirs('jaffle_shop/models/staging', exist_ok=True)
os.makedirs('jaffle_shop/models/marts', exist_ok=True)
print("디렉토리 생성 완료")

In [ ]:
%%writefile jaffle_shop/dbt_project.yml
name: 'jaffle_shop'
version: '1.0.0'
profile: 'jaffle_shop'

model-paths: ["models"]

In [ ]:
%%writefile jaffle_shop/profiles.yml
jaffle_shop:
  target: dev
  outputs:
    dev:
      type: duckdb
      path: /content/warehouse.duckdb

## Silver 모델 작성 (stg_ 접두어 = staging = Silver)

각 Bronze 테이블에 대응하는 Silver 모델을 만든다.

- 타입 캐스팅
- 컬럼명 표준화
- 단위 변환

In [ ]:
%%writefile jaffle_shop/models/staging/stg_orders.sql
with source as (
    select * from bronze.orders
),

cleaned as (
    select
        id as order_id,
        user_id as customer_id,
        cast(order_date as date) as order_date,
        status
    from source
)

select * from cleaned

In [ ]:
%%writefile jaffle_shop/models/staging/stg_customers.sql
with source as (
    select * from bronze.customers
),

cleaned as (
    select
        id as customer_id,
        first_name,
        last_name
    from source
)

select * from cleaned

In [ ]:
%%writefile jaffle_shop/models/staging/stg_payments.sql
with source as (
    select * from bronze.payments
),

cleaned as (
    select
        id as payment_id,
        order_id,
        payment_method,
        amount / 100.0 as amount_dollars
    from source
)

select * from cleaned

## dbt 실행

In [ ]:
!cd jaffle_shop && dbt run --select staging

## 결과 확인

Silver 레이어가 제대로 만들어졌는지 확인한다.

In [ ]:
conn = duckdb.connect('warehouse.duckdb')

# stg_orders: order_date가 DATE로 바뀌었는가
print("=== stg_orders ===")
display(conn.execute("SELECT * FROM stg_orders LIMIT 5").fetchdf())

print("\n=== 타입 확인 ===")
display(conn.execute("""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'stg_orders'
""").fetchdf())

In [ ]:
# stg_payments: amount가 달러 단위로 변환되었는가
print("=== stg_payments ===")
display(conn.execute("SELECT * FROM stg_payments LIMIT 5").fetchdf())

In [ ]:
# stg_customers
print("=== stg_customers ===")
display(conn.execute("SELECT * FROM stg_customers LIMIT 5").fetchdf())

## Bronze vs Silver 비교

Bronze와 Silver를 나란히 놓고 차이를 확인한다.

In [ ]:
print("=== Bronze (원본) ===")
display(conn.execute("SELECT id, user_id, order_date, status FROM bronze.orders LIMIT 3").fetchdf())

print("\n=== Silver (정제) ===")
display(conn.execute("SELECT * FROM stg_orders LIMIT 3").fetchdf())

print("\n변경 사항:")
print("- id → order_id (명확한 이름)")
print("- user_id → customer_id (의미 통일)")
print("- order_date: VARCHAR → DATE (타입 캐스팅)")

In [ ]:
print("=== Bronze payments (센트) ===")
display(conn.execute("SELECT id, order_id, payment_method, amount FROM bronze.payments LIMIT 3").fetchdf())

print("\n=== Silver payments (달러) ===")
display(conn.execute("SELECT * FROM stg_payments LIMIT 3").fetchdf())

print("\n변경 사항:")
print("- id → payment_id (명확한 이름)")
print("- amount(센트) → amount_dollars(달러) (단위 변환 + 이름 변경)")

In [ ]:
# 정리
conn.close()